This main jupyter notebook of ENLR, which is about the spatial extension of the extended narrow lines (ENLRs). We used the data from MaNGA, the massive integrate field unit(IFU) observation, to selected the AGNs with extended narrow line regions and to revisit the size luminosity relation of ENLR.

**to-do list**
<form>
  <input type="checkbox" checked> Changing the structure of the pipeline<br>
  <input type="checkbox" checked> Excluding binary galaxies from the sample.<br>
  <input type="checkbox" checked> Comparing your sample with PSF and the radius of the major axies of [OIII] to check whether the ENLR was resolved.<br>
  <input type="checkbox" checked> Applying dust extinction before calculating the ENLR radius<br>
  <input type="checkbox" checked> Using MC method to get the mearsuring error of all the results.<br>
  <input type="checkbox" > Checking the size of ENLR with the 5500Å to determine the check the whether the relation is due to the AGN or the host galaxy<br> 
  <input type="checkbox" checked> Including previous results<br>
  <input type="checkbox" > Checking the morphology of ENLR with the b/a of the galaxy to find possible relation with related with egde-on galaxies<br>
  <input type="checkbox" > Getting the stellar mass of the galaxies to check the ENLR go with L[OIII]/Mass_stellar (L/M as a roughly accretion rate).<br>
  <input type="checkbox"> What's the relation with hubble morphology related?<br>
  <input type="checkbox"> Determining the ionizing state of the gas. Using the software developed by Thomas2018 <a href="https://github.com/ADThomas-astro/NebulaBayes">NebulaBayes</a><br> 
  <input type="checkbox"> Determining the difference between bulge-dominanted and disk-dominanted galaxies, using the cataloge of Meer2015<br>
  <input type="checkbox" checked> Determining the gas density and inonized state of the NLR according MaNGA internal Nascimento2018. <i>the S/N was not enough for that</i><br>
  <input type="checkbox" checked> Cross-matching with all sky surveys (ROSAT, WISE, FIRST)<br>
  <input type="checkbox"> Using some indicator that can show the evolution of the ENLR, age? metallicity? SFH?<br>
  <input type="checkbox"> Picking out the outliers of the sample, finding the possible reasons that contribute the scatter.<br>
  <input type="checkbox" checked> Whether this relation is due to secular process? Single or multiple process? something like the [‘red geysers] (Cheung 2016)<br>
  <input type="checkbox" checked> Using the line ratio that can stand for the ionization ([O III]/Hb) to see whether the results is consistant with OIII. <i>limited by resolution</i><br>
</form>

**Contents**

The structure of the whole jupyter notebook

- [Sample Selection](./Sample_selection.ipynb) 
- [Methods](./Methods.ipynb)
- [Extended Narrow line Region](./ENLR.ipynb)
- [Results](./Results.ipynb)
- [Discussion](./Discussion.ipynb)

**Figures**

The shortcuts linking to the plots showed in the paper

- [Figure 1](./Methods.ipynb#Demo-Sample-Detail)
- [Figure 2](./Sample_selection.ipynb#All-the-sample-in-BPT-diagram)
- [Figure 3](./Methods.ipynb#Testting-for-a-single-galaxy)
- [Figure 4](./ENLR.ipynb#Compare-IFU-and-longslit)
- [Figure 5](./ENLR.ipynb#Demo-Different-PSF-Effects)
- [Figure 6](./Results.ipynb#Size(ENLR)---Luminosty(AGN)-relation)
- [Figure 7](./Discussion.ipynb#IR-luminosity)
- Figure 8

In [ ]:
# Read all the computed data, generate the table in paper
# It is also easy to modify the code to generate the all the data into files
# History:
#   2018-11-12: first release
#   2019-06-26: updated for re-submission

# the columns of the table
# ---------------------------------------------------------------------------------
# plateifu, mangaid, Ra, Dec, z, g_PSF, logL_[O III], R15, R_16, resolved, outflow_strength, radio_loudness

import numpy as np
from astropy.table import Table

# used for generate table for paper
table_for_paper = True
line_split = 40

this_AGN = Table.read('data/AGNs-2019-04-26.dat', format='ascii')
this_Ldata = Table.read('data/LO3-2019-06-25.dat', format="ascii")
this_Rdata = Table.read('data/R_fsb-(10)-2019-06-19.dat', format='ascii')
this_Rdata_100 = Table.read('data/R_fsb-(100)-2019-06-19.dat', format='ascii')
this_Lv_first = np.loadtxt('data/Lv_first.txt')
this_loudness = np.loadtxt('data/loudness.txt')
this_L_IR = np.loadtxt('data/L_IR.txt')


LO3 = this_Ldata['LO3_redcorr']
LO3_err = 0.5 * (this_Ldata['LO3_redcorr_err_up'].data + this_Ldata['LO3_redcorr_err_low'].data)

R16 = 10**this_Rdata['logR_fsb2_orig'].data / 1000
R16_err = this_Rdata['logR_fsb2_orig_err'].data * 10**this_Rdata['logR_fsb2_orig'].data / 1000 * np.log(10)
resolved = (this_Rdata['is_limit'].data == 0)
R16[~resolved] = this_Rdata['logR_fsb1'].data[~resolved]

R15 = 10**this_Rdata_100['logR_fsb2_orig'].data / 1000
R15_err = this_Rdata_100['logR_fsb2_orig_err'].data * 10**this_Rdata_100['logR_fsb2_orig'].data / 1000 * np.log(10)
resolved_100 = (this_Rdata_100['is_limit'].data == 0)
R15[~resolved_100] = np.nan
R15_err[~resolved_100] = np.nan

outflow_strength = this_Ldata['outflow_strength']
radio_loudness = this_loudness

tot_num = len(this_AGN[:])
for i in range(tot_num):
    plateifu = this_AGN['plateifu'][i]
    mangaid = this_AGN['mangaid'][i]
    ra = this_AGN['ra'][i]
    dec = this_AGN['dec'][i]
    z = this_AGN['z'][i]
    psf = this_AGN['psf'][i]
    
    if np.isnan(R15[i]) or np.isnan(R15_err[i]) or (R15[i] < 0.01):
        R15_string = 'nan'
    else:
        R15_string = "{:.2f}$\pm${:.2f}".format(R15[i], R15_err[i])
        
    if np.isnan(R16[i]) or np.isnan(R16_err[i]) or (R16[i] < 0.01):
        R16_string = 'nan'
    else:
        R16_string = "{:.2f}$\pm${:.2f}".format(R16[i], R16_err[i])
    if this_Lv_first[i] < 0:
        Lv_first = np.nan
    else:
        Lv_first = this_Lv_first[i]
    
    print("{} & {:.6f} & {:.6f} & {} & {:.2f} & {:.2f}$\pm${:.2f} & {:.2f} & {:.2f} & {} & {} & {} & {:.2f} \\\\".format(
        plateifu, ra, dec, z, psf, LO3[i], LO3_err[i], this_L_IR[i], Lv_first, R15_string, R16_string, resolved[i], radio_loudness[i]))
    if not (i+1)%40:
        print("------------------------------\n\n\n\n---------------------------")